In [1]:
from sage.all import *
from hashlib import sha256, sha1
from Crypto.Util.number import *
from pwn import *
from ecdsa.ecdsa import Private_key, Public_key, generator_256
import re
import json

In [2]:
io = remote("14.225.211.39", 13375)
request_1 = {"cmd": "get_transcript"}
request_2 = {"cmd": "registrar_submission",
             "msg": "Student: h114mx001, Grade: A",
             "pubkey": None,
             "r": None,
             "s": None}
# for integer catch
pattern = re.compile(r"(\d+)")
# non-sense messages
io.sendline(json.dumps(request_1).encode())
for _ in range(8):
    print(io.recvline())
# leaked_percentage
leaked_percentage = int(pattern.findall(io.recvline().decode())[0])
leaked_bits = int(leaked_percentage * 256 / 100)
B = 2 ** leaked_bits
print(leaked_percentage)
print(io.recvline())
msg1 = json.loads(io.recvline().decode())
msg2 = json.loads(io.recvline().decode())
msg3 = json.loads(io.recvline().decode())
print(io.recvline())
pubkey = io.recvline().decode().strip()
# convert to tuple int from (x, y) str
pubkey = tuple(map(int, pattern.findall(pubkey)))

[x] Opening connection to 14.225.211.39 on port 13375
[x] Opening connection to 14.225.211.39 on port 13375: Trying 14.225.211.39
[+] Opening connection to 14.225.211.39 on port 13375: Done
b'Welcome to College of VHC!\n'
b'Hell yeah, we are the professors grading your exam!\n'
b'Professor s4shaNull is/are grading your exam...\n'
b'Your grade is F. We are 54% confident about that.\n'
b'Professor goldenskygiang is/are grading your exam...\n'
b'Your grade is F. We are 55% confident about that.\n'
b'Professor Thanh-WuTan & Cl4ris is/are grading your exam...\n'
b'Your grade is F. We are 65% confident about that.\n'
58
b'We use curve secp256k1.\n'
b'Now submit your transcript to Office of Registrar! Remember to tell them reverify our signature with this pubkey.\n'


In [3]:
known = [msg1, msg2, msg3]
msgs = []
sigs = []

for msg in known:
    msgs_hsh = bytes_to_long(sha256(msg['msg'].encode()).digest())
    msgs.append(msgs_hsh)
    msg_sigs = (int(msg['r'], 16), int(msg['s'], 16))
    sigs.append(msg_sigs)

In [4]:
# curve init 
# https://neuromancer.sk/std/nist/P-256
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
K = GF(p)
a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
E = EllipticCurve(K, (a, b))
G = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)
n = E.order()
P = E(pubkey)

In [5]:
def make_matrix():
    (r1, s1), (r2, s2), (r3, s3) = sigs
    m1, m2, m3 = msgs

    t1 = r1 * inverse_mod(s1, n)
    t2 = r2 * inverse_mod(s2, n)
    t3 = r3 * inverse_mod(s3, n)

    a1 = m1 * inverse_mod(s1, n)
    a2 = m2 * inverse_mod(s2, n)
    a3 = m3 * inverse_mod(s3, n)

    basis = [ [n,           0,           0,            0,                  0],
              [0,           n,           0,            0,                  0],
              [0,           0,           n,            0,                  0],
              [t1,          t2,          t3,           B / n,              0],
              [a1,          a2,          a3,           0,                  B]
             ]
    return Matrix(QQ, basis)

In [6]:
matrix = make_matrix()
new_matrix = matrix.LLL(early_red=True, use_siegel=True)
r1_inv = pow(sigs[0][0], -1, n)
s1 = sigs[0][1]
d = 0
for row in new_matrix:
    nonce = int(row[0])
    potential_private_key = int(r1_inv * ((s1 * nonce) - msgs[0]))
    if G * potential_private_key == P:
        print("Found private key: ", potential_private_key)
        d = potential_private_key
        break
print(d)

Found private key:  83762756493044608548654129447419464879663577301422681408816629019118533514356
83762756493044608548654129447419464879663577301422681408816629019118533514356


In [7]:
# craft message
G = generator_256
pk = Public_key(G, G * d)
sk = Private_key(pk, d)
msg = "Student: h114mx001, GPA: 4.0"
msg_hash = sha256(msg.encode()).digest()
# you can use any nonce
nonce = sha256(b"hehehehe").digest()
signature = sk.sign(bytes_to_long(msg_hash), bytes_to_long(nonce))
payload = {"msg": msg, "pubkey": (int(pk.point.x()), int(pk.point.y())),"r": hex(signature.r), "s": hex(signature.s)}
print(payload)

{'msg': 'Student: h114mx001, GPA: 4.0', 'pubkey': (7713807504938394601069661789016955879040234882672017235376729571675452480162, 3111693241484916818383684507700202118658495127033153567444754653936442398088), 'r': '0x3e7b1498143f9c8d0066295f7cbee7bbaefe2d632478db757f708e4a75402da8', 's': '0xd3413215b7349c885bea5c469743aa8ec2f9c56e3ed5c763abc95534f977751a'}


In [8]:
request_2["msg"] = payload["msg"]
request_2["pubkey"] = payload["pubkey"]
request_2["r"] = payload["r"]
request_2["s"] = payload["s"]
print(request_2)
io.sendline(json.dumps(request_2).encode())
print(io.recvline())
print(io.recvline())
print(io.recvline())

{'cmd': 'registrar_submission', 'msg': 'Student: h114mx001, GPA: 4.0', 'pubkey': (7713807504938394601069661789016955879040234882672017235376729571675452480162, 3111693241484916818383684507700202118658495127033153567444754653936442398088), 'r': '0x3e7b1498143f9c8d0066295f7cbee7bbaefe2d632478db757f708e4a75402da8', 's': '0xd3413215b7349c885bea5c469743aa8ec2f9c56e3ed5c763abc95534f977751a'}
b'Hello, this is Office of Registrar. We are verifying your transcript...\n'
b"Congrats! You are getting President's List this academic year!\n"
b"Here is your flag: VHC2023{d0n't_try_th1s_4t_h0me}\n"


In [ ]:
test = {"cmd": "registrar_submission", "msg": "Student: h114mx001, GPA: 4.0", "pubkey": (17946873503476711239924231345164312800696993624013434822027562686555764449690, 78795712735901101180555795177608565505810033906111059955498467848732713081335), "r": 0x67492a0ded84d9525f8f4c906c3441b3b77e646712b8617495117d5a45b9c084, "s": 0xf90cb68a0cd4eab1e853e3d2b5d1aaffbb50c98aaa4c67673a2d14ed2114970e}